In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from torch_geometric.nn import GCNConv, BatchNorm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import optuna
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_recall_curve, auc

# Enhanced GNN Model
class EnhancedParkinsonGCN(nn.Module):
    def __init__(self, hidden_channels, fc_neurons, num_layers, dropout_rate):
        super(EnhancedParkinsonGCN, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_rate)

        # Input layer
        self.layers.append(GCNConv(22, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers
        for _ in range(num_layers - 1):
            self.layers.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_channels, fc_neurons)
        self.fc2 = nn.Linear(fc_neurons, 1)  # Binary classification output

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply GCN layers with batch normalization
        for i in range(self.num_layers):
            x = self.layers[i](x, edge_index)
            x = torch.relu(x)
            x = self.batch_norms[i](x)
            x = self.dropout(x)

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)).view(-1)  # Binary classification, reshape output to match the target size
        return x

# Function to run training and validation
def train_and_evaluate(model, optimizer, criterion, train_graph, val_graph, y_val_tensor, device, epochs):
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(train_graph)
        loss = criterion(output, y_train_tensor.to(device).float())
        loss.backward()
        optimizer.step()

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_output = model(val_graph)
            y_val_pred = (val_output > 0.5).float()
            val_acc = accuracy_score(y_val_tensor.cpu(), y_val_pred.cpu())
            if val_acc > best_val_acc:
                best_val_acc = val_acc

    return best_val_acc

# Define the objective function for Optuna optimization
def objective(trial):
    hidden_channels = trial.suggest_int('hidden_channels', 64, 256, step=64)
    fc_neurons = trial.suggest_int('fc_neurons', 64, 256, step=64)
    num_layers = trial.suggest_int('num_layers', 2, 4)  # Number of GCN layers
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # Instantiate the model
    model = EnhancedParkinsonGCN(hidden_channels=hidden_channels, fc_neurons=fc_neurons,
                                 num_layers=num_layers, dropout_rate=dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCELoss()

    # Move model to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_graph.to(device)
    val_graph.to(device)

    # Training and Validation
    best_val_acc = train_and_evaluate(model, optimizer, criterion, train_graph, val_graph, y_test_tensor, device, epochs=50)

    return best_val_acc

# Load Dataset and Prepare Data (Same as earlier)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data'
df = pd.read_csv(url)

# Drop 'name' column and separate features (X) and target (y)
X = df.drop(columns=['name', 'status'])
y = df['status']

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Create a graph: X_train as nodes and their pairwise connections based on correlation
correlation_matrix = np.corrcoef(X_train_scaled.T)
edge_index = np.array(np.nonzero(correlation_matrix > 0.7))

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_resampled.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create PyTorch Geometric graphs for training and testing
train_graph = Data(x=X_train_tensor, edge_index=torch.tensor(edge_index, dtype=torch.long))
val_graph = Data(x=X_test_tensor, edge_index=torch.tensor(edge_index, dtype=torch.long))

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=3600)

# Best hyperparameters
print(f"Best hyperparameters: {study.best_params}")

# Instantiate the best model
best_model = EnhancedParkinsonGCN(hidden_channels=study.best_params['hidden_channels'],
                                  fc_neurons=study.best_params['fc_neurons'],
                                  num_layers=study.best_params['num_layers'],
                                  dropout_rate=study.best_params['dropout_rate'])

# Train and Evaluate Best Model on the test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)
train_graph.to(device)
val_graph.to(device)

optimizer = optim.Adam(best_model.parameters(), lr=study.best_params['lr'], weight_decay=study.best_params['weight_decay'])
criterion = nn.BCELoss()

# Train the best model on the entire training dataset
train_and_evaluate(best_model, optimizer, criterion, train_graph, val_graph, y_test_tensor, device, epochs=100)

# Test the model and evaluate performance
best_model.eval()
with torch.no_grad():
    output = best_model(val_graph)
    y_pred = (output > 0.5).float()

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred.cpu().numpy()))

    # ROC AUC and Precision-Recall AUC
    roc_auc = roc_auc_score(y_test, output.cpu().numpy())
    precision, recall, _ = precision_recall_curve(y_test, output.cpu().numpy())
    pr_auc = auc(recall, precision)

    print(f"Accuracy: {accuracy_score(y_test, y_pred.cpu().numpy()):.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")


[I 2024-10-19 19:35:23,005] A new study created in memory with name: no-name-b067309c-cf17-464c-811c-a87b489d853d
/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_11472/2692181998.py:78: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_11472/2692181998.py:79: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
[I 2024-10-19 19:35:23,158] Trial 0 finished with value: 0.8717948717948718 and parameters: {'hidden_channels': 192, 'fc_neurons': 192, 'num_layers': 3, 'dropout_rate': 0.4358991572256036, '

Best hyperparameters: {'hidden_channels': 128, 'fc_neurons': 128, 'num_layers': 4, 'dropout_rate': 0.31853756865995664, 'lr': 0.0007944140565778424, 'weight_decay': 1.4003574671207357e-05}

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.93      0.97      0.95        29

    accuracy                           0.92        39
   macro avg       0.91      0.88      0.90        39
weighted avg       0.92      0.92      0.92        39

Accuracy: 0.9231
ROC AUC: 0.9259
PR AUC: 0.9700


In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from torch_geometric.nn import GCNConv, BatchNorm
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import optuna
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_recall_curve, auc
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Enhanced GNN Model with Residual Connections
class ResidualParkinsonGCN(nn.Module):
    def __init__(self, hidden_channels, fc_neurons, num_layers, dropout_rate):
        super(ResidualParkinsonGCN, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_rate)

        # Input layer
        self.layers.append(GCNConv(22, hidden_channels))
        self.batch_norms.append(BatchNorm(hidden_channels))

        # Hidden layers with skip (residual) connections
        for _ in range(num_layers - 1):
            self.layers.append(GCNConv(hidden_channels, hidden_channels))
            self.batch_norms.append(BatchNorm(hidden_channels))

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_channels, fc_neurons)
        self.fc2 = nn.Linear(fc_neurons, 1)  # Binary classification output

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        input_x = x  # Keep the input for residual connections

        # Apply GCN layers with batch normalization and residual connections
        for i in range(self.num_layers):
            residual = x
            x = self.layers[i](x, edge_index)
            x = torch.relu(x)
            x = self.batch_norms[i](x)
            x = self.dropout(x)

            # Residual connection
            if i > 0:
                x += residual  # Add skip connection

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)).view(-1)  # Binary classification, reshape output to match the target size
        return x

# Function to run training and validation
def train_and_evaluate(model, optimizer, criterion, scheduler, train_graph, val_graph, y_val_tensor, device, epochs):
    best_val_acc = 0
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        output = model(train_graph)
        loss = criterion(output, y_train_tensor.to(device).float())
        loss.backward()
        optimizer.step()

        # Learning rate scheduling
        scheduler.step(loss)

        # Validation phase
        model.eval()
        with torch.no_grad():
            val_output = model(val_graph)
            y_val_pred = (val_output > 0.5).float()
            val_acc = accuracy_score(y_val_tensor.cpu(), y_val_pred.cpu())
            if val_acc > best_val_acc:
                best_val_acc = val_acc

    return best_val_acc

# Define the objective function for Optuna optimization
def objective(trial):
    hidden_channels = trial.suggest_int('hidden_channels', 64, 256, step=64)
    fc_neurons = trial.suggest_int('fc_neurons', 64, 256, step=64)
    num_layers = trial.suggest_int('num_layers', 2, 5)  # Number of GCN layers
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # Instantiate the model
    model = ResidualParkinsonGCN(hidden_channels=hidden_channels, fc_neurons=fc_neurons,
                                 num_layers=num_layers, dropout_rate=dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCELoss()

    # Scheduler to reduce learning rate on plateau
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)

    # Move model to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_graph.to(device)
    val_graph.to(device)

    # Training and Validation
    best_val_acc = train_and_evaluate(model, optimizer, criterion, scheduler, train_graph, val_graph, y_test_tensor, device, epochs=50)

    return best_val_acc

# Load Dataset and Prepare Data (Same as earlier)
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data'
df = pd.read_csv(url)

# Drop 'name' column and separate features (X) and target (y)
X = df.drop(columns=['name', 'status'])
y = df['status']

# Train-test split with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

# Create a graph: X_train as nodes and their pairwise connections based on correlation
correlation_matrix = np.corrcoef(X_train_scaled.T)
edge_index = np.array(np.nonzero(correlation_matrix > 0.7))

# Convert to torch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_resampled.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# Create PyTorch Geometric graphs for training and testing
train_graph = Data(x=X_train_tensor, edge_index=torch.tensor(edge_index, dtype=torch.long))
val_graph = Data(x=X_test_tensor, edge_index=torch.tensor(edge_index, dtype=torch.long))

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=3600)

# Best hyperparameters
print(f"Best hyperparameters: {study.best_params}")

# Instantiate the best model
best_model = ResidualParkinsonGCN(hidden_channels=study.best_params['hidden_channels'],
                                  fc_neurons=study.best_params['fc_neurons'],
                                  num_layers=study.best_params['num_layers'],
                                  dropout_rate=study.best_params['dropout_rate'])

# Train and Evaluate Best Model on the test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)
train_graph.to(device)
val_graph.to(device)

optimizer = optim.Adam(best_model.parameters(), lr=study.best_params['lr'], weight_decay=study.best_params['weight_decay'])
criterion = nn.BCELoss()

# Scheduler for learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)

# Train the best model on the entire training dataset
train_and_evaluate(best_model, optimizer, criterion, scheduler, train_graph, val_graph, y_test_tensor, device, epochs=100)

# Test the model and evaluate performance
best_model.eval()
with torch.no_grad():
    output = best_model(val_graph)
    y_pred = (output > 0.5).float()

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred.cpu().numpy()))

    # ROC AUC and Precision-Recall AUC
    roc_auc = roc_auc_score(y_test, output.cpu().numpy())
    precision, recall, _ = precision_recall_curve(y_test, output.cpu().numpy())
    pr_auc = auc(recall, precision)

    print(f"Accuracy: {accuracy_score(y_test, y_pred.cpu().numpy()):.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")


[I 2024-10-19 19:38:04,033] A new study created in memory with name: no-name-1c924b87-af14-4c7e-ab6e-08f72a42e573
/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_11472/3312789977.py:88: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_11472/3312789977.py:89: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
/opt/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings

Best hyperparameters: {'hidden_channels': 192, 'fc_neurons': 64, 'num_layers': 4, 'dropout_rate': 0.40474365447122473, 'lr': 0.0007094144485562472, 'weight_decay': 8.704062739211758e-06}

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.93      0.97      0.95        29

    accuracy                           0.92        39
   macro avg       0.91      0.88      0.90        39
weighted avg       0.92      0.92      0.92        39

Accuracy: 0.9231
ROC AUC: 0.9121
PR AUC: 0.9632


In [25]:
from torch_geometric.nn import GATConv

# Enhanced GNN Model using GAT
class AttentionParkinsonGAT(nn.Module):
    def __init__(self, hidden_channels, fc_neurons, num_layers, heads, dropout_rate):
        super(AttentionParkinsonGAT, self).__init__()
        self.layers = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout_rate)

        # Input layer with GAT
        self.layers.append(GATConv(22, hidden_channels, heads=heads))
        self.batch_norms.append(BatchNorm(hidden_channels * heads))

        # Hidden layers with GAT and residual connections
        for _ in range(num_layers - 1):
            self.layers.append(GATConv(hidden_channels * heads, hidden_channels, heads=heads))
            self.batch_norms.append(BatchNorm(hidden_channels * heads))

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_channels * heads, fc_neurons)
        self.fc2 = nn.Linear(fc_neurons, 1)  # Binary classification output

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Apply GAT layers with batch normalization
        for i in range(self.num_layers):
            residual = x
            x = self.layers[i](x, edge_index)
            x = torch.relu(x)
            x = self.batch_norms[i](x)
            x = self.dropout(x)
            
            # Add skip connection after first layer
            if i > 0:
                x += residual  # Skip (residual) connection

        # Fully connected layers
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)).view(-1)  # Binary classification, reshape output to match the target size
        return x

# Define the objective function for Optuna optimization with GAT
def objective(trial):
    hidden_channels = trial.suggest_int('hidden_channels', 64, 256, step=64)
    fc_neurons = trial.suggest_int('fc_neurons', 64, 256, step=64)
    num_layers = trial.suggest_int('num_layers', 2, 4)
    heads = trial.suggest_int('heads', 2, 8)  # Number of attention heads
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.5)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)

    # Instantiate the GAT model
    model = AttentionParkinsonGAT(hidden_channels=hidden_channels, fc_neurons=fc_neurons,
                                  num_layers=num_layers, heads=heads, dropout_rate=dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCELoss()

    # Scheduler to reduce learning rate on plateau
    scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5, verbose=True)

    # Move model to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_graph.to(device)
    val_graph.to(device)

    # Training and Validation
    best_val_acc = train_and_evaluate(model, optimizer, criterion, scheduler, train_graph, val_graph, y_test_tensor, device, epochs=50)

    return best_val_acc

# Run Optuna optimization with GAT
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=3600)

# Best hyperparameters
print(f"Best hyperparameters: {study.best_params}")

# Instantiate the best GAT model
best_model = AttentionParkinsonGAT(hidden_channels=study.best_params['hidden_channels'],
                                  fc_neurons=study.best_params['fc_neurons'],
                                  num_layers=study.best_params['num_layers'],
                                  heads=study.best_params['heads'],
                                  dropout_rate=study.best_params['dropout_rate'])

# Train and Evaluate Best GAT Model on the test data
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)
train_graph.to(device)
val_graph.to(device)

optimizer = optim.Adam(best_model.parameters(), lr=study.best_params['lr'], weight_decay=study.best_params['weight_decay'])
criterion = nn.BCELoss()

# Scheduler for learning rate
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=25, factor=0.5, verbose=True)

# Train the best model on the entire training dataset
train_and_evaluate(best_model, optimizer, criterion, scheduler, train_graph, val_graph, y_test_tensor, device, epochs=100)

# Test the model and evaluate performance
best_model.eval()
with torch.no_grad():
    output = best_model(val_graph)
    y_pred = (output > 0.5).float()

    # Classification Report
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred.cpu().numpy()))

    # ROC AUC and Precision-Recall AUC
    roc_auc = roc_auc_score(y_test, output.cpu().numpy())
    precision, recall, _ = precision_recall_curve(y_test, output.cpu().numpy())
    pr_auc = auc(recall, precision)

    print(f"Accuracy: {accuracy_score(y_test, y_pred.cpu().numpy()):.4f}")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"PR AUC: {pr_auc:.4f}")


[I 2024-10-19 19:41:26,013] A new study created in memory with name: no-name-44259d8f-1fc4-4cb2-8459-67fb608397c3
/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_11472/4226720914.py:52: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
/var/folders/b7/fvm8bd317t9f9_j96tkt0q800000gn/T/ipykernel_11472/4226720914.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
/opt/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings

Best hyperparameters: {'hidden_channels': 128, 'fc_neurons': 256, 'num_layers': 3, 'heads': 8, 'dropout_rate': 0.23311603191133481, 'lr': 0.00018406038607752612, 'weight_decay': 1.3075844169802353e-05}

Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.93      0.97      0.95        29

    accuracy                           0.92        39
   macro avg       0.91      0.88      0.90        39
weighted avg       0.92      0.92      0.92        39

Accuracy: 0.9231
ROC AUC: 0.9586
PR AUC: 0.9857
